In [1]:
from tensorflow import keras


model1 = keras.models.load_model('../models/pre_model_1_1_10_epochs.h5')
model2 = keras.models.load_model('../models/pre_model_2_1.h5')
model3 = keras.models.load_model('../models/pre_model_3_1_10_epochs.h5')
model4 = keras.models.load_model('../models/pre_model_4_1_10_epochs.h5')
model5 = keras.models.load_model('../models/pre_model_5_1_10_epochs.h5')
model6 = keras.models.load_model('../models/pre_model_6_1_20_2_epochs.h5')
model7 = keras.models.load_model('../models/pre_model_7_1_10_epochs.h5')
model8 = keras.models.load_model('../models/pre_model_8_1_10_epochs.h5')


In [3]:
all_models = [model1,model2, model3,model4,model5,model6,model7,model8]
rule_names = rules = [
            "Kein Scooter",
            "Scooter unvollständig/zu wenig Umgebung",
            "Scooter steht zu nah an Straße (Bordsteinkante)",
            "Scooter steht nicht frei",
            "Scooter auf Grünfläche",
            "Scooter blockiert Fahrradständer",
            "Scooter steht an ÖPNV Haltestelle / Parkverbo",
            "Scooter steht in Einfahrt / im Weg"
        ]

In [4]:
from PIL import Image
import numpy as np
# Function to load and resize an image
def load_image(img_path):
    with Image.open(img_path) as img:
        img = img.resize((512, 512))
        img = img.convert('RGB')  # Ensure image is in RGB format
        return np.array(img)


In [5]:
import matplotlib.pyplot as plt

def show_img(img_array):
    img_array.astype('uint8')
    plt.imshow(img_array)
    plt.axis('off')  # Hide the axes
    plt.show()

In [6]:
import os
import random

folder_path = "../Yoio_Park_Proof"
all_files = os.listdir(folder_path)
random_image_file = random.choice(all_files)
img_path = os.path.join(folder_path, random_image_file)
img = load_image(img_path)
img_array = np.expand_dims(img, axis=0)
print(img.shape)

(512, 512, 3)


In [7]:
import os
import random

folder_path = "../Yoio_Park_Proof"
all_files = os.listdir(folder_path)
random_image_file = random.choice(all_files)
img_path = os.path.join(folder_path, random_image_file)

def predict_image(models,img_path):


    img = load_image(img_path)

    for model in models:
        img_array = np.expand_dims(img, axis=0)
        predictions = model.predict(img_array)
        
        #Model 4 predicted Falsch herum i guess
        if models.index(model) !=5 :
            #if predictions[0][0] > predictions[0][1]: Wäre eine Option
            if predictions[0][0]>0.95:
                print("Verstoß")
                print(rules[models.index(model)])
                break
            else:
                print("Richtig")
        else: 
            if predictions[0][1]>0.95:
                print("Verstoß")
                print(rules[models.index(model)])
                break
            else:
                print("Richtig")


    show_img(img)
    

In [ ]:
#predict_image(models=all_models)

In [8]:
import pandas as pd
balanced_labels = pd.read_csv('../all_models_combine/chosen_labels_all.csv', sep=';',index_col=False)
balanced_labels.sample(5)

,Image_name,Rule1,Rule2,Rule3,Rule4,Rule5,Rule6,Rule7,Rule8
954,16056471509c397ab796b7ee675aa7125bbf873c8ec2fe...,0,0,0,0,0,1,0,0
212,1586008937b2a07c1b41862ccca5c684237e42d40cb7ba...,0,0,0,0,0,0,0,0
351,160925040226de200781276a21642a324571953590df34...,1,0,0,0,0,0,0,0
57,1609833838929e05cd1cef5dd261b013e433ad8d284677...,0,0,0,0,0,0,0,0
285,1599936330ff50cf2d974525ba46b83f6ed4b49d04bd0e...,1,0,0,0,0,0,0,0


In [9]:

columns=["Image_name","Rule1","Rule2","Rule3","Rule4","Rule5","Rule6","Rule7","Rule8"]
predictions_df = pd.DataFrame(columns=columns)

i=0
while i <= 1000:
    folder_path = "../Yoio_Park_Proof/"
    img_name=str(balanced_labels.sample(1).iloc[0,0])


    img = load_image(folder_path+img_name)

    predictions_array =[img_name]

    for model in all_models:
        img_array = np.expand_dims(img, axis=0)
        predictions = model.predict(img_array)
        #Model 4 predicted Falsch herum i guess
        if all_models.index(model) !=5 :
            #if predictions[0][0] > predictions[0][1]: Wäre eine Option
            if predictions[0][0]>0.95:
                int_pred=1
            else:
                int_pred=0
        else: 
            if predictions[0][1]>0.95:
                int_pred=1
            else:
                int_pred=0
        predictions_array.append(int_pred)

    predictions_df.loc[len(predictions_df)] = predictions_array
    #new_row = pd.DataFrame(predictions_array, columns=columns)
    #predictions_df = pd.concat([predictions_array,new_row],ignore_index=True)

    i += 1

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 981ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 983ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 971ms/step


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 976ms/step


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 939ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 966ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 954ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 66ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 73ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step
1/1 ━━━━━

In [10]:
print(predictions_df.sample(10))

                                            Image_name  Rule1  Rule2  Rule3  \
116  16052623768bc08bc20bfaaad883e68b243f56e9fd7af1...      0      1      0   
628  1611326022fa620f9d216a61c2b60230c18ae15050ab08...      0      1      0   
399  16022695926a80e3731a9a915db2b7dbd5cb6294dd368b...      0      1      0   
285  1598030283daf277bf179bc86fe3b5cb5ca02cfbb093d9...      0      0      0   
839  16135496336f9d376dcd30d28a7e55c5eaedaecce0e01a...      0      1      0   
645  159784330767c607f52bcc3657d5791da29a0e27a6bc3c...      0      1      0   
487  15835314128302c134a0982308e6d305327932bd39a653...      0      1      0   
669  160742292658bf483f34feaac5041b53e5323d9113a30b...      0      0      0   
838  16013796632e342f03396dafca0e116bfcd44f4afd7fc4...      0      0      0   
48   15967283712cde6d5568b6597e4ae025066ea02c98c8a3...      0      1      0   

     Rule4  Rule5  Rule6  Rule7  Rule8  
116      0      1      0      0      0  
628      0      0      0      0      1  
399    

In [11]:
predictions_df.to_csv('../all_models_combine/predicted_labels_all_2.csv', sep=';', index=False)